In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()
spark

In [5]:
categories = spark.read.parquet("../../data/sklep/categories")
categories.printSchema()

root
 |-- category_id: integer (nullable = true)
 |-- category_department_id: integer (nullable = true)
 |-- category_name: string (nullable = true)


In [11]:
customers = spark.read.parquet("../../data/sklep/customers")
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_password: string (nullable = true)
 |-- customer_street: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)


In [12]:
customers.show(5)

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|           00725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [13]:
departments = spark.read.parquet("../../data/sklep/departments")
departments.printSchema()
departments.show(5)

root
 |-- department_id: integer (nullable = true)
 |-- department_name: string (nullable = true)

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|            2|        Fitness|
|            3|       Footwear|
|            4|        Apparel|
|            5|           Golf|
|            6|       Outdoors|
+-------------+---------------+
only showing top 5 rows


In [15]:
order_items = spark.read.parquet("../../data/sklep/order_items")
order_items.printSchema()
order_items.toPandas().head(5)

root
 |-- order_item_id: integer (nullable = true)
 |-- order_item_order_id: integer (nullable = true)
 |-- order_item_product_id: integer (nullable = true)
 |-- order_item_quantity: integer (nullable = true)
 |-- order_item_subtotal: float (nullable = true)
 |-- order_item_product_price: float (nullable = true)


,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
0,1,1,957,1,299.980011,299.980011
1,2,2,1073,1,199.990005,199.990005
2,3,2,502,5,250.000000,50.000000
3,4,2,403,1,129.990005,129.990005
4,5,4,897,2,49.980000,24.990000


In [16]:
orders = spark.read.parquet("../../data/sklep/orders")
orders.printSchema()
orders.head(5)

root
 |-- order_id: integer (nullable = true)
 |-- order_date: long (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)


[Row(order_id=1, order_date=1374735600000, order_customer_id=11599, order_status='CLOSED'),
 Row(order_id=2, order_date=1374735600000, order_customer_id=256, order_status='PENDING_PAYMENT'),
 Row(order_id=3, order_date=1374735600000, order_customer_id=12111, order_status='COMPLETE'),
 Row(order_id=4, order_date=1374735600000, order_customer_id=8827, order_status='CLOSED'),
 Row(order_id=5, order_date=1374735600000, order_customer_id=11318, order_status='COMPLETE')]

In [17]:
products = spark.read.parquet("../../data/sklep/products")
products.printSchema()
products.head(5)

root
 |-- product_id: integer (nullable = true)
 |-- product_category_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_description: string (nullable = true)
 |-- product_price: float (nullable = true)
 |-- product_image: string (nullable = true)


[Row(product_id=1, product_category_id=2, product_name='Quest Q64 10 FT. x 10 FT. Slant Leg Instant U', product_description='', product_price=59.97999954223633, product_image='http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy'),
 Row(product_id=2, product_category_id=2, product_name="Under Armour Men's Highlight MC Football Clea", product_description='', product_price=129.99000549316406, product_image='http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat'),
 Row(product_id=3, product_category_id=2, product_name="Under Armour Men's Renegade D Mid Football Cl", product_description='', product_price=89.98999786376953, product_image='http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat'),
 Row(product_id=4, product_category_id=2, product_name="Under Armour Men's Renegade D Mid Football Cl", product_description='', product_price=89.98999786376953, product_image='http://images.acmesports.sports/Unde

In [18]:
join_types = ["inner", "cross", "outer", "full", "left_outer", "right_outer", "left_semi", "left_anti"]

In [20]:
categories.join(departments, categories.category_department_id == departments.department_id).show(5)

+-----------+----------------------+-------------------+-------------+---------------+
|category_id|category_department_id|      category_name|department_id|department_name|
+-----------+----------------------+-------------------+-------------+---------------+
|          1|                     2|           Football|            2|        Fitness|
|          1|                     2|           Football|            2|        Fitness|
|          2|                     2|             Soccer|            2|        Fitness|
|          2|                     2|             Soccer|            2|        Fitness|
|          3|                     2|Baseball & Softball|            2|        Fitness|
+-----------+----------------------+-------------------+-------------+---------------+


In [22]:
for join_type in join_types:
    cats_on_deps = categories.join(other=departments, on=categories.category_department_id == departments.department_id, how=join_type)
    count = cats_on_deps.count()
    print(join_type, count)

inner 192
cross 192
outer 212
full 212
left_outer 212
right_outer 192


In [23]:
order_items.join(products, order_items.order_item_product_id == products.product_id).show(5)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|       957|                 43|Diamondback Women...|                   |       299.98|http://images.acm...|
|            1|                  1|                  957|                  1|   

In [24]:
spark

In [27]:
customers.createOrReplaceTempView("customers")

In [29]:
df = spark.sql("SELECT * FROM customers")

In [30]:
df.show(5)

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|           00725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [32]:
df.toPandas().head(5)

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,00725


In [33]:
data_frames = [categories, departments, order_items, orders, products]
data_frames_names = ["categories", "departments", "order_items", "orders", "products"]
for df, df_name in zip(data_frames, data_frames_names):
    df.createOrReplaceTempView(df_name)

In [34]:
spark.sql("select * from categories").show(5)

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
+-----------+----------------------+-------------------+


In [35]:
spark.sql("select category_id, category_name from categories").show(5)

+-----------+-------------------+
|category_id|      category_name|
+-----------+-------------------+
|          1|           Football|
|          2|             Soccer|
|          3|Baseball & Softball|
|          4|         Basketball|
|          5|           Lacrosse|
+-----------+-------------------+


In [37]:
spark.sql("select distinct category_id, category_name from categories where category_id = 1").show(5)

+-----------+-------------+
|category_id|category_name|
+-----------+-------------+
|          1|     Football|
+-----------+-------------+


In [42]:
with open("../../src/sql/select_category.sql", "r") as f:
    category_query_template = f.read()


In [43]:
category_query_template

'select distinct category_id, category_name from categories where category_id = {category_id}\n'

In [44]:
category_query_template.format(category_id=1)

'select distinct category_id, category_name from categories where category_id = 1\n'

In [45]:
spark.sql(category_query_template.format(category_id=1)).show(5)

+-----------+-------------+
|category_id|category_name|
+-----------+-------------+
|          1|     Football|
+-----------+-------------+


In [47]:
def get_query_template(query_name: str) -> str:
    with open(f"../../src/sql/{query_name}.sql", "r") as f:
        return f.read()

def get_query(query_name: str, **kwargs) -> str:
    return get_query_template(query_name).format(**kwargs)

In [48]:
spark.sql(get_query("select_category", category_id=1)).show(5)

+-----------+-------------+
|category_id|category_name|
+-----------+-------------+
|          1|     Football|
+-----------+-------------+


In [50]:
orders.show(5)

+--------+-------------+-----------------+---------------+
|order_id|   order_date|order_customer_id|   order_status|
+--------+-------------+-----------------+---------------+
|       1|1374735600000|            11599|         CLOSED|
|       2|1374735600000|              256|PENDING_PAYMENT|
|       3|1374735600000|            12111|       COMPLETE|
|       4|1374735600000|             8827|         CLOSED|
|       5|1374735600000|            11318|       COMPLETE|
+--------+-------------+-----------------+---------------+


In [52]:
orders.select("order_status").distinct().show()

+---------------+
|   order_status|
+---------------+
|PENDING_PAYMENT|
|       COMPLETE|
|        ON_HOLD|
| PAYMENT_REVIEW|
|     PROCESSING|
|         CLOSED|
|SUSPECTED_FRAUD|
|        PENDING|
|       CANCELED|
+---------------+


In [53]:
spark.sql("select distinct order_status from orders").show()

+---------------+
|   order_status|
+---------------+
|PENDING_PAYMENT|
|       COMPLETE|
|        ON_HOLD|
| PAYMENT_REVIEW|
|     PROCESSING|
|         CLOSED|
|SUSPECTED_FRAUD|
|        PENDING|
|       CANCELED|
+---------------+


In [54]:
df = spark.sql("select distinct order_status from orders")
df

DataFrame[order_status: string]

In [55]:
df.collect()

[Row(order_status='PENDING_PAYMENT'),
 Row(order_status='COMPLETE'),
 Row(order_status='ON_HOLD'),
 Row(order_status='PAYMENT_REVIEW'),
 Row(order_status='PROCESSING'),
 Row(order_status='CLOSED'),
 Row(order_status='SUSPECTED_FRAUD'),
 Row(order_status='PENDING'),
 Row(order_status='CANCELED')]

In [56]:
df.collect()[0]

Row(order_status='PENDING_PAYMENT')

In [57]:
df.collect()[0].order_status

'PENDING_PAYMENT'

In [59]:
df.collect()[0]["order_status"]

'PENDING_PAYMENT'

In [60]:
df.collect()[0][0]

'PENDING_PAYMENT'

In [61]:
df.collect()[0].asDict()

{'order_status': 'PENDING_PAYMENT'}

In [63]:
df = spark.sql("select distinct order_status from orders order by order_status")
df.show()

+---------------+
|   order_status|
+---------------+
|       CANCELED|
|         CLOSED|
|       COMPLETE|
|        ON_HOLD|
| PAYMENT_REVIEW|
|        PENDING|
|PENDING_PAYMENT|
|     PROCESSING|
|SUSPECTED_FRAUD|
+---------------+


In [87]:
orders.select("order_status").distinct().orderBy("order_status").show()

ConnectionRefusedError: [Errno 61] Connection refused

In [65]:
query = """
select c.category_name, count(order_item_quantity) as count
from order_items oi
inner join products p on oi.order_item_product_id = p.product_id
inner join categories c on c.category_id = p.product_category_id
group by c.category_name
order by count desc
limit 10;
"""

In [66]:
spark.sql(query).show()

+--------------------+------+
|       category_name| count|
+--------------------+------+
|              Cleats|196408|
|      Men's Footwear|177968|
|     Women's Apparel|168280|
|Indoor/Outdoor Games|154384|
|             Fishing|138600|
|        Water Sports|124320|
|    Camping & Hiking|109832|
|    Cardio Equipment| 99896|
|       Shop By Sport| 87872|
|         Electronics| 25248|
+--------------------+------+


In [69]:
query = get_query("top_categories")
query

'select c.category_name, count(order_item_quantity) as count\nfrom order_items oi\ninner join products p on oi.order_item_product_id = p.product_id\ninner join categories c on c.category_id = p.product_category_id\ngroup by c.category_name\norder by count desc\nlimit 10;\n'

In [70]:
spark.sql(query).show()

+--------------------+------+
|       category_name| count|
+--------------------+------+
|              Cleats|196408|
|      Men's Footwear|177968|
|     Women's Apparel|168280|
|Indoor/Outdoor Games|154384|
|             Fishing|138600|
|        Water Sports|124320|
|    Camping & Hiking|109832|
|    Cardio Equipment| 99896|
|       Shop By Sport| 87872|
|         Electronics| 25248|
+--------------------+------+


In [71]:
query = """
select
    oi.order_item_product_id,
    sum(cast(oi.order_item_subtotal as float)) as revenue
from order_items oi
inner join orders o on oi.order_item_order_id = o.order_id
where o.order_status <> 'CANCELED' and o.order_status <> 'SUSPECTED_FRAUD'
group by order_item_product_id;
"""

In [72]:
spark.sql(query).show()

+---------------------+------------------+
|order_item_product_id|           revenue|
+---------------------+------------------+
|                  897| 78568.55933380127|
|                  858| 44797.76123046875|
|                  251|  74511.7201538086|
|                  804| 71963.99922943115|
|                   78| 72392.75991821289|
|                  642|          106800.0|
|                   44| 218843.5263671875|
|                  743| 41477.56134033203|
|                  860|     19199.6796875|
|                  926|56220.839210510254|
|                  822|149152.92210388184|
|                  625| 47197.64129638672|
|                   93|  79668.1192703247|
|                  924| 53342.63919830322|
|                  725|           31104.0|
|                  671| 52077.52136230469|
|                  305|           51740.0|
|                  906| 87065.15906524658|
|                  797|61525.799255371094|
|                  777| 64951.87976074219|
+----------

In [80]:
spark

In [86]:
categories.write.mode("overwrite").parquet("../../data/categories.parquet")

In [85]:
customers.write.mode("overwrite").csv("../../data/customers.csv")